In [1]:
from pathlib import Path
import geopandas as gpd
import json

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter
from toolbox_continu_inzicht.inspections.filter import Filter
from shapely.geometry import LineString
from toolbox_continu_inzicht.inspections.inspections import ClassifyInspections

In [2]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-inspectie-hhnk.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

In [3]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

# HHNK inspectie voorbeeld lines

In [4]:
# lees csv met inspecties en filter inspecties
filter = Filter(data_adapter=data_adapter)
filter.run(input="inspectie_lines_csv", output="inspectie_lines_filtered_csv")

In [5]:
# sla inspecties op als geojson
df_in = data_adapter.input("inspectie_lines_filtered_csv")
df_in["line"] = df_in.apply(
    lambda row: LineString(
        [
            (row["EXT_MIN_X"], row["EXT_MIN_Y"]),
            (row["INSIDE_X"], row["INSIDE_Y"]),
            (row["EXT_MAX_X"], row["EXT_MAX_Y"]),
        ]
    ),
    axis=1,
)

# lijst met kolommen die niet in de geojson moeten komen
drop_columns = [
    "EXT_MAX_X",
    "EXT_MAX_Y",
    "EXT_MIN_X",
    "EXT_MIN_Y",
    "INSIDE_X",
    "INSIDE_Y",
    "SHAPE_Length",
]

df_in = df_in.drop(columns=drop_columns)

gdf_in = gpd.GeoDataFrame(df_in, geometry="line")
gdf_in = gdf_in.set_crs(epsg=28992)
gdf_in = gdf_in.to_crs(epsg=4326)

gdf_in.columns = gdf_in.columns.str.lower().str.replace(" ", "_")
gdf_in = gdf_in.rename(columns={"foto_url": "image_url"})
gdf_in = gdf_in.rename(columns={"foto_omgeving_url": "image_url_omgeving"})

output_file = Path(config.global_variables["rootdir"]) / Path(
    config.data_adapters["inspectie_lines_filtered_geojson"]["path"]
)

gdf_in.to_file(output_file, driver="GeoJSON")

In [6]:
# lees gefilterde inspecties en classificeer ze
classify_inspections = ClassifyInspections(data_adapter=data_adapter)
classify_inspections.run(
    input=["inspectie_lines_filtered_geojson", "inspectie_lines_styling_csv"],
    output=[
        "inspectie_lines_filtered_styled_geojson",
        "inspectie_lines_styled_legenda_csv",
    ],
)

In [7]:
gdf_in = data_adapter.input("inspectie_lines_filtered_styled_geojson")
data_adapter.output("inspectie_lines_filtered_styled_ci_to_database", gdf_in)

In [8]:
df_in = data_adapter.input("inspectie_lines_styled_legenda_csv")
df_in.fillna("", inplace=True)

df = data_adapter.input("inspectie_lines_layer_csv")
df["group_name"] = "HHNK Inspecties"
df["layer_name"] = "Steenbekleding (lijnen)"
df["layer_legend"] = json.dumps([value for key, value in df_in.T.to_dict().items()])
df["layer_table"] = config.data_adapters[
    "inspectie_lines_filtered_styled_ci_to_database"
]["table"]
df["layer_type"] = "table"
data_adapter.output("inspectie_layers_ci_to_database", df)

# HHNK inspectie voorbeeld points

In [9]:
# lees csv met inspecties en filter inspecties
filter = Filter(data_adapter=data_adapter)
filter.run(input="inspectie_points_csv", output="inspectie_points_filtered_csv")

In [10]:
# sla inspecties op als geojson
df_in = data_adapter.input("inspectie_points_filtered_csv")
geometry = gpd.points_from_xy(df_in["POINT_X"], df_in["POINT_Y"])

# lijst met kolommen die niet in de geojson moeten komen
drop_columns = ["POINT_X", "POINT_Y"]

df_in = df_in.drop(columns=drop_columns)

gdf_in = gpd.GeoDataFrame(df_in, geometry=geometry)
gdf_in = gdf_in.set_crs(epsg=28992)
gdf_in = gdf_in.to_crs(epsg=4326)

gdf_in.columns = gdf_in.columns.str.lower().str.replace(" ", "_")
gdf_in = gdf_in.rename(columns={"foto_url": "image_url"})
gdf_in = gdf_in.rename(columns={"foto_omgeving_url": "image_url_omgeving"})

output_file = Path(config.global_variables["rootdir"]) / Path(
    config.data_adapters["inspectie_points_filtered_geojson"]["path"]
)

gdf_in.to_file(output_file, driver="GeoJSON")

In [11]:
# lees gefilterde inspecties en classificeer ze
classify_inspections = ClassifyInspections(data_adapter=data_adapter)
classify_inspections.run(
    input=["inspectie_points_filtered_geojson", "inspectie_points_styling_csv"],
    output=[
        "inspectie_points_filtered_styled_geojson",
        "inspectie_points_styled_legenda_csv",
    ],
)

In [12]:
gdf_in = data_adapter.input("inspectie_points_filtered_styled_geojson")
data_adapter.output("inspectie_points_filtered_styled_ci_to_database", gdf_in)

In [13]:
df_in = data_adapter.input("inspectie_points_styled_legenda_csv")
df_in.fillna("", inplace=True)

df = data_adapter.input("inspectie_points_layer_csv")
df["group_name"] = "HHNK Inspecties"
df["layer_name"] = "Steenbekleding (punten)"
df["layer_legend"] = json.dumps([value for key, value in df_in.T.to_dict().items()])
df["layer_table"] = config.data_adapters[
    "inspectie_points_filtered_styled_ci_to_database"
]["table"]
df["layer_type"] = "table"
data_adapter.output("inspectie_layers_ci_to_database", df)